In [31]:
import psycopg2

# Connexion à la base de données source (base1)
conn1 = psycopg2.connect(host="172.20.12.13", dbname="data_externe", user="hroussaffa", password="mcot")
cursor1 = conn1.cursor()

# Exécuter la requête SQL pour récupérer les données de la première table
cursor1.execute("SELECT objectid FROM mos.mos2014_pn_ps_pi where c_2014_n3 in ('321','220','2221','311','240')")
rows1 = cursor1.fetchall()

# Connexion à la base de données destination (base2)
conn2 = psycopg2.connect(host="172.20.12.13", dbname="oeil_traitement", user="hroussaffa", password="mcot")
cursor2 = conn2.cursor()




In [32]:

# Exécuter la requête SQL pour récupérer les données de la deuxième table
cursor2.execute('SELECT id_thematique FROM "occupation_sol"."faits_foncier_mos_formation_arboree_dafe_2014"')
rows2 = cursor2.fetchall()

In [23]:
cursor1.execute("""
    CREATE TABLE IF NOT EXISTS "mos"."missing_data_table" (
        objectid INTEGER
    )
""")


In [33]:
# Identifier les lignes de la première table qui n'ont pas de correspondance dans la deuxième table
missing_rows = [row1 for row1 in rows1 if row1 not in rows2]

In [34]:
# Insérer les lignes manquantes dans une nouvelle table
for row in missing_rows:
    cursor1.execute("INSERT INTO mos.missing_data_table (objectid) VALUES (%s)", (row[0],))

In [ ]:
cursor1.execute("""
CREATE VIEW mos.missing_data_result AS
SELECT mos.*
FROM mos.missing_data_table AS missing
INNER JOIN mos.mos2014_pn_ps_pi AS mos ON missing.objectid = mos.objectid;

ALTER TABLE mos.missing_data_result
    OWNER TO oeil_admin;

GRANT SELECT ON TABLE mos.missing_data_result TO mos_ro;
GRANT ALL ON TABLE mos.missing_data_result TO mos_rw;
GRANT ALL ON TABLE mos.missing_data_result TO oeil_admin;
""")


In [35]:

# Valider les changements et fermer les connexions
conn1.commit()
conn2.commit()
cursor1.close()
conn1.close()
cursor2.close()
conn2.close()